In [10]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [11]:
column_names = list(range(0,13))
column_names.append('Classification')

df = pd.read_csv("wine.data", names = column_names) 
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,Classification
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


In [12]:
df.dtypes

0                 float64
1                 float64
2                 float64
3                 float64
4                   int64
5                 float64
6                 float64
7                 float64
8                 float64
9                 float64
10                float64
11                float64
12                  int64
Classification      int64
dtype: object

In [13]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
7                 0
8                 0
9                 0
10                0
11                0
12                0
Classification    0
dtype: int64

In [14]:
df.nunique()

0                 126
1                 133
2                  79
3                  63
4                  53
5                  97
6                 132
7                  39
8                 101
9                 132
10                 78
11                122
12                121
Classification      3
dtype: int64

In [15]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df['Classification'] = label_encoder.fit_transform(df['Classification'])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,Classification
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2


In [16]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (13,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
5/5 [==============================] - 2s 179ms/step - loss: 24.1852 - accuracy: 0.3750 - precision: 0.3750 - recall: 0.3750 - val_loss: 28.3342 - val_accuracy: 0.2222 - val_precision: 0.2222 - val_recall: 0.2222
Epoch 2/50
5/5 [==============================] - 0s 13ms/step - loss: 14.7614 - accuracy: 0.3938 - precision: 0.3938 - recall: 0.3938 - val_loss: 11.2049 - val_accuracy: 0.6111 - val_precision: 0.6111 - val_recall: 0.6111
Epoch 3/50
5/5 [==============================] - 0s 12ms/step - loss: 15.0566 - accuracy: 0.5813 - precision: 0.5813 - recall: 0.5813 - val_loss: 12.3497 - val_accuracy: 0.5556 - val_precision: 0.5556 - val_recall: 0.5556
Epoch 4/50
5/5 [==============================] - 0s 14ms/step - loss: 8.0447 - accuracy: 0.5625 - precision: 0.5633 - recall: 0.5562 - val_loss: 9.0999 - val_accuracy: 0.2222 - val_precision: 0.2222 - val_recall: 0.2222
Epoch 5/50
5/5 [==============================] - 0s 14ms/step - loss: 7.0904 - accuracy: 0.4500 - precision:

Epoch 38/50
5/5 [==============================] - 0s 11ms/step - loss: 0.8387 - accuracy: 0.7312 - precision: 0.7434 - recall: 0.7063 - val_loss: 0.7701 - val_accuracy: 0.7778 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 39/50
5/5 [==============================] - 0s 11ms/step - loss: 0.6750 - accuracy: 0.8438 - precision: 0.8535 - recall: 0.8375 - val_loss: 0.6820 - val_accuracy: 0.8333 - val_precision: 0.8125 - val_recall: 0.7222
Epoch 40/50
5/5 [==============================] - 0s 10ms/step - loss: 0.6177 - accuracy: 0.8687 - precision: 0.8710 - recall: 0.8438 - val_loss: 0.5557 - val_accuracy: 0.7778 - val_precision: 0.8235 - val_recall: 0.7778
Epoch 41/50
5/5 [==============================] - 0s 10ms/step - loss: 0.6230 - accuracy: 0.8250 - precision: 0.8387 - recall: 0.8125 - val_loss: 0.8925 - val_accuracy: 0.7222 - val_precision: 0.7222 - val_recall: 0.7222
Epoch 42/50
5/5 [==============================] - 0s 10ms/step - loss: 0.7401 - accuracy: 0.7750 - precision: 0

5/5 [==============================] - 0s 15ms/step - loss: 0.4197 - accuracy: 0.8750 - precision: 0.8846 - recall: 0.8625 - val_loss: 0.2174 - val_accuracy: 0.8333 - val_precision: 0.8824 - val_recall: 0.8333
Epoch 10/50
5/5 [==============================] - 0s 14ms/step - loss: 0.3117 - accuracy: 0.8938 - precision: 0.9045 - recall: 0.8875 - val_loss: 0.1197 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 11/50
5/5 [==============================] - 0s 14ms/step - loss: 0.3918 - accuracy: 0.8375 - precision: 0.8471 - recall: 0.8313 - val_loss: 0.2867 - val_accuracy: 0.8333 - val_precision: 0.9375 - val_recall: 0.8333
Epoch 12/50
5/5 [==============================] - 0s 15ms/step - loss: 0.4143 - accuracy: 0.8750 - precision: 0.8910 - recall: 0.8687 - val_loss: 1.0421 - val_accuracy: 0.8333 - val_precision: 0.8333 - val_recall: 0.8333
Epoch 13/50
5/5 [==============================] - 0s 15ms/step - loss: 0.8526 - accuracy: 0.7812 - precision: 0.7812 - reca

Epoch 29/50
5/5 [==============================] - 0s 16ms/step - loss: 1.1756 - accuracy: 0.8000 - precision: 0.8038 - recall: 0.7937 - val_loss: 0.6119 - val_accuracy: 0.7778 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 30/50
1/1 [==============================] - 0s 44ms/step
Epoch 1/50
5/5 [==============================] - 0s 29ms/step - loss: 1.3959 - accuracy: 0.7125 - precision: 0.7152 - recall: 0.7063 - val_loss: 2.1188 - val_accuracy: 0.5556 - val_precision: 0.5556 - val_recall: 0.5556
Epoch 2/50
5/5 [==============================] - 0s 12ms/step - loss: 0.7338 - accuracy: 0.8125 - precision: 0.8165 - recall: 0.8062 - val_loss: 0.7674 - val_accuracy: 0.7778 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 3/50
5/5 [==============================] - 0s 13ms/step - loss: 0.3277 - accuracy: 0.8875 - precision: 0.8868 - recall: 0.8813 - val_loss: 0.5469 - val_accuracy: 0.7778 - val_precision: 0.8235 - val_recall: 0.7778
Epoch 4/50
5/5 [==============================] - 0

5/5 [==============================] - 0s 15ms/step - loss: 0.2715 - accuracy: 0.9000 - precision: 0.9057 - recall: 0.9000 - val_loss: 0.2337 - val_accuracy: 0.9444 - val_precision: 0.9444 - val_recall: 0.9444
Epoch 14/50
5/5 [==============================] - 0s 14ms/step - loss: 0.4409 - accuracy: 0.8438 - precision: 0.8438 - recall: 0.8438 - val_loss: 0.7271 - val_accuracy: 0.7222 - val_precision: 0.8125 - val_recall: 0.7222
Epoch 15/50
5/5 [==============================] - 0s 15ms/step - loss: 0.6001 - accuracy: 0.8375 - precision: 0.8375 - recall: 0.8375 - val_loss: 0.1879 - val_accuracy: 0.8889 - val_precision: 0.8889 - val_recall: 0.8889
Epoch 16/50
1/1 [==============================] - 0s 33ms/step
Epoch 1/50
5/5 [==============================] - 0s 34ms/step - loss: 0.6042 - accuracy: 0.8375 - precision: 0.8428 - recall: 0.8375 - val_loss: 0.3300 - val_accuracy: 0.8889 - val_precision: 0.8889 - val_recall: 0.8889
Epoch 2/50
5/5 [==============================] - 0s 13ms/ste

5/5 [==============================] - 0s 13ms/step - loss: 0.6851 - accuracy: 0.8313 - precision: 0.8313 - recall: 0.8313 - val_loss: 1.9425 - val_accuracy: 0.6111 - val_precision: 0.6471 - val_recall: 0.6111
Epoch 9/50
5/5 [==============================] - 0s 15ms/step - loss: 1.3601 - accuracy: 0.7750 - precision: 0.7785 - recall: 0.7688 - val_loss: 0.4081 - val_accuracy: 0.7778 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 10/50
5/5 [==============================] - 0s 19ms/step - loss: 0.9017 - accuracy: 0.8500 - precision: 0.8500 - recall: 0.8500 - val_loss: 0.9942 - val_accuracy: 0.7778 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 11/50
5/5 [==============================] - 0s 17ms/step - loss: 0.6280 - accuracy: 0.8375 - precision: 0.8375 - recall: 0.8375 - val_loss: 1.6716 - val_accuracy: 0.7778 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 12/50
5/5 [==============================] - 0s 15ms/step - loss: 0.9040 - accuracy: 0.8375 - precision: 0.8365 - recal

6/6 [==============================] - 0s 16ms/step - loss: 0.4754 - accuracy: 0.8882 - precision: 0.8882 - recall: 0.8882 - val_loss: 1.0813 - val_accuracy: 0.8824 - val_precision: 0.8824 - val_recall: 0.8824
Epoch 12/50
6/6 [==============================] - 0s 15ms/step - loss: 0.3868 - accuracy: 0.9255 - precision: 0.9255 - recall: 0.9255 - val_loss: 2.5029 - val_accuracy: 0.7647 - val_precision: 0.7647 - val_recall: 0.7647
Epoch 13/50
6/6 [==============================] - 0s 17ms/step - loss: 5.8077 - accuracy: 0.5217 - precision: 0.5217 - recall: 0.5217 - val_loss: 2.3393 - val_accuracy: 0.5882 - val_precision: 0.5882 - val_recall: 0.5882
Epoch 14/50
6/6 [==============================] - 0s 15ms/step - loss: 2.0537 - accuracy: 0.7081 - precision: 0.7081 - recall: 0.7081 - val_loss: 1.0052 - val_accuracy: 0.8824 - val_precision: 0.8824 - val_recall: 0.8824
Epoch 15/50
6/6 [==============================] - 0s 21ms/step - loss: 0.9272 - accuracy: 0.7950 - precision: 0.7925 - reca

In [18]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  0.7751633986928106
Average Precision Score:  0.7751633986928106
Average Recall Score:  0.7751633986928106
Average F1-Score:  0.7751633986928106
